In [21]:
from typing import List
from enum import Enum, auto
import random

In [22]:
WORLD_SIZE_x = 4
WORLD_SIZE_y = 4

In [23]:
class Percept():
    time_step: int
    bump: bool
    breeze: bool
    stench: bool
    scream: bool
    glitter: bool
    reward: int
    done: bool
        
    def __init__(self, time_step: int, bump: bool, breeze: bool, stench: bool, scream: bool, glitter: bool, reward: int, done: bool):
        # add code to set the instance variables of the percept
        self.time_step = time_step
        self.bump = bump
        self.breeze = breeze
        self.stench = stench
        self.scream = scream
        self.glitter = glitter
        self.reward = reward
        self.done = done

    def __str__(self):
        # add helper function to return the contents of a percept in a readable form
        return f"Percepts: Time Step={self.time_step}, Bump={self.bump}, Breeze={self.breeze}, Stench={self.stench}, Scream={self.scream}, Glitter={self.glitter}, Reward={self.reward}, Done={self.done}"

In [24]:
class Action(Enum):
    LEFT = 0
    RIGHT = 1
    FORWARD = 2
    GRAB = 3
    SHOOT = 4
    CLIMB = 5

In [25]:
class Orientation(Enum):
    E = 0
    S = 1
    W = 2
    N = 3

    def __str__(self) -> str:
        # code for function to return the letter code ("E", "S", etc.) of this instance of an orientation
        # You could create a __str__(self) for this instead of the symbol function if you prefer
        return self.name
    
    def turn_right(self) -> 'Orientation':
        # return a new orientation turned right
        # Note: the quotes around the type Orientation are because of a quirk in Python.  You can't refer
        # to Orientation without quotes until it is defined (and we are in the middle of defining it)
        if self == Orientation.E:
            return Orientation.S
        elif self == Orientation.S:
            return Orientation.W
        elif self == Orientation.W:
            return Orientation.N
        elif self == Orientation.N:
            return Orientation.E
        
    def turn_left(self) -> 'Orientation':
        # return a new orientation turned left
        if self == Orientation.E:
            return Orientation.N
        elif self == Orientation.N:
            return Orientation.W
        elif self == Orientation.W:
            return Orientation.S
        elif self == Orientation.S:
            return Orientation.E

In [26]:
class Location:
    x: int
    y: int
        
    def __init__(self, x: int, y: int):
        self.x = x
        self.y = y
        
    def __str__(self):
        return f'({self.x}, {self.y})'
    
    def __repr__(self) -> str:
        return f'({self.x}, {self.y})'
    
    def __eq__(self, other):
        if not isinstance(other, Location):
            return False
        return self.x == other.x and self.y == other.y
    
    def is_left_of(self, location: 'Location')->bool:
    # return True if self is just left of given location
        return self.x == location.x - 1 and self.y == location.y
   
    def is_right_of(self, location: 'Location')->bool:
    # return True if self is just right of given location  
        return self.x == location.x + 1 and self.y == location.y
    
    def is_above(self, location: 'Location')->bool:
    # return True if self is immediately above given location    
        return self.x == location.x and self.y == location.y+1
    
    def is_below(self, location: 'Location')->bool:
    # return True if self is immediately below given location   
        return self.x == location.x and self.y+1 == location.y
    
    def neighbours(self)->List['Location']:
    # return list of neighbour locations    
        neighbours = []
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            x, y = self.x + dx, self.y + dy
            if (1 <= x <= WORLD_SIZE_x) and (1 <= y <= WORLD_SIZE_y):
                neighbours.append(Location(x, y))
        return neighbours
    
    # return True if location given is self's location
    def is_location(self, location: 'Location')->bool:
        return self.x == location.x and self.y == location.y

    def at_left_edge(self) -> bool:
    # return True if at the left edge of the grid   
        return self.x == 1
    
    def at_right_edge(self) -> bool:
    # return True if at the right edge of the grid  
        return self.x == WORLD_SIZE_x
    
    def at_top_edge(self) -> bool:
    # return True if at the top edge of the grid    
        return self.y == WORLD_SIZE_y
   
    def at_bottom_edge(self) -> bool:
     # return True if at the bottom edge of the grid    
        return self.y == 1
    
    def forward(self, orientation) -> bool:
    # modify self.x and self.y to reflect a forward move and return True if bumped a wall  
        if orientation == Orientation.N:
            if self.at_top_edge():
                return True
            else:
                self.y = self.y+1
                return False
        elif orientation == Orientation.S:
            if self.at_bottom_edge():
                return True
            else:
                self.y = self.y-1
                return False
        elif orientation == Orientation.E:
            if self.at_right_edge():
                return True
            else:
                self.x = self.x+1
                return False
        elif orientation == Orientation.W:
            if self.at_left_edge():
                return True
            else:
                self.x = self.x-1
                return False

    def set_to(self, location: 'Location'):
    # set self.x and self.y to the given location  
        self.x = location.x
        self.y = location.y
    
    @staticmethod
    def from_linear(n: int) -> 'Location':
    # convert an index from 0 to 15 to a location
        x = (n % WORLD_SIZE_x) + 1
        y = (n // WORLD_SIZE_y) + 1
        return Location (x, y)
    
    
    def to_linear(self)->int:
    # convert self to an index from 0 to 15  
        return (self.y - 1) * WORLD_SIZE_y + (self.x - 1)  
    
    
    @staticmethod
    def random() -> 'Location':
        # return a random location   

        x = random.randint(1, WORLD_SIZE_x)
        y = random.randint(1, WORLD_SIZE_y)

        while x == 1 and y == 1:
            x = random.randint(1, WORLD_SIZE_x)
            y = random.randint(1, WORLD_SIZE_y) 

        return Location(x, y)

In [39]:
class Environment:
    wumpus_location: Location
    wumpus_alive: bool
    agent_location: Location
    agent_orientation: Orientation
    agent_has_arrow: bool
    agent_has_gold: bool
    game_over: bool
    gold_location: Location
    pit_locations: List[Location]
    time_step: int
        
    def __init__(self, width: int, height: int, allow_climb_without_gold: bool, pit_prob: float):
        # initialize the environment state variables (use make functions below)
        self.width = width
        self.height = height
        self.allow_climb_without_gold = allow_climb_without_gold
        self.pit_prob = pit_prob

        self.wumpus_location, self.wumpus_alive = self.make_wumpus()
        self.agent_location = Location(1,1)
        self.gold_location = self.make_gold()
        self.pit_locations = self.make_pits(self.pit_prob)
        self.agent_orientation = Orientation['E']
        self.agent_has_arrow = True
        self.agent_has_gold = False
        self.game_over = False
        self.time_step = 0
        
        
    def make_wumpus(self): 
        # choose a random location for the wumpus (not bottom left corner) and set it to alive
        while True:
            location = Location.random()
            if location != Location(1,1):  # Ensure it's not the bottom left corner
                self.wumpus_location = location
                self.wumpus_alive = True
                break
        return self.wumpus_location, self.wumpus_alive

    def make_gold(self):
        # choose a random location for the gold (not bottom left corner)
        while True:
            location = Location.random()
            if location != Location(1,1):  # Ensure it's not the bottom left corner
                self.gold_location = location
                break
        return self.gold_location
        
    def make_pits(self, pit_prob: float):
        # create pits with prob pit_prob for all locations except the bottom left corner
        self.pit_locations = []
        for x in range(1, self.width+1):
            for y in range(1, self.height+1):
                if (x, y) != (1, 1):  # Exclude bottom left corner
                    if random.random() < pit_prob:
                        self.pit_locations.append(Location(x, y))
        return self.pit_locations
        
    def is_pit_at(self, location: Location) -> bool:
        # return true if there is a pit at location
        return location in self.pit_locations
        
    def is_pit_adjacent_to_agent(self) -> bool:
        # return true if there is a pit above, below, left or right of agent's current location
        for pit in self.pit_locations:
            if pit in self.agent_location.neighbours():
                return True
        return False
                
    def is_wumpus_adjacent_to_agent(self) -> bool:
        # return true if there is a wumpus adjacent to the agent
        if self.wumpus_location in self.agent_location.neighbours():
                return True
        return False
        
    def is_agent_at_hazard(self)->bool:
        # return true if the agent is at the location of a pit or the wumpus
        return (self.agent_location == self.wumpus_location and self.wumpus_alive==True) or self.is_pit_at(self.agent_location)
        
    def is_wumpus_at(self, location: Location) -> bool:
        # return true if there is a wumpus at the given location
        return self.wumpus_location == location
        
    def is_agent_at(self, location: Location) -> bool:
        # return true if the agent is at the given location
        return self.agent_location == location
        
    def is_gold_at(self, location: Location) -> bool:
        # return true if the gold is at the given location
        return self.gold_location == location
        
    def is_glitter(self) -> bool:
        # return true if the agent is where the gold is
        return self.agent_location == self.gold_location
        
    def is_breeze(self) -> bool:
        # return true if one or pits are adjacent to the agent or the agent is in a room with a pit
        if self.is_pit_adjacent_to_agent():
            return True
        if self.is_pit_at(self.agent_location):
            return True
        return False
    
    def is_stench(self) -> bool:
        # return true if the wumpus is adjacent to the agent or the agent is in the room with the wumpus
        if self.is_wumpus_adjacent_to_agent():
            return True
        if self.is_wumpus_at(self.agent_location):
            return True
        return False

    def wumpus_in_line_of_fire(self) -> bool:
        # return true if the wumpus is a cell the arrow would pass through if fired
        if self.agent_orientation == Orientation['E']:
            if (self.agent_location.x < self.wumpus_location.x) and (self.agent_location.y == self.wumpus_location.y):
                return True
        if self.agent_orientation == Orientation['W']:
            if (self.agent_location.x > self.wumpus_location.x) and (self.agent_location.y == self.wumpus_location.y):
                return True
        if self.agent_orientation == Orientation['N']:
            if (self.agent_location.x == self.wumpus_location.x) and (self.agent_location.y < self.wumpus_location.y):
                return True
        if self.agent_orientation == Orientation['S']:
            if (self.agent_location.x == self.wumpus_location.x) and (self.agent_location.y > self.wumpus_location.y):
                return True
        else:
            return False


    def kill_attempt(self) -> bool:
        # return true if the wumpus is alive and in the line of fire
        # if so set the wumpus to dead
        if self.wumpus_in_line_of_fire() and self.wumpus_alive and self.agent_has_arrow:
            self.wumpus_alive = False
        self.agent_has_arrow = False
        return self.wumpus_alive
        
    def step(self, action: Action) -> Percept:
        # for each of the actions, make any agent state changes that result and return a percept including the reward
        # First, update agent state based on action
        if action == Action.LEFT:
            # Turn agent left
            self.agent_orientation = self.agent_orientation.turn_left()
            return Percept(time_step=self.time_step+1, bump=False, breeze=self.is_breeze(), stench=self.is_stench(), scream=False, glitter=self.is_glitter(), reward=-1, done=self.game_over)

        elif action == Action.RIGHT:
            # Turn agent right
            self.agent_orientation = self.agent_orientation.turn_right()
            return Percept(time_step=self.time_step+1, bump=False, breeze=self.is_breeze(), stench=self.is_stench(), scream=False, glitter=self.is_glitter(), reward=-1, done=self.game_over)
    
        elif action == Action.FORWARD:
            # Move agent forward
            bump = self.agent_location.forward(self.agent_orientation)
            reward = -1
            if self.is_agent_at_hazard():
                reward -= 1000
                self.game_over = True
            if self.agent_has_gold == True:
                self.gold_location = self.agent_location
            return Percept(time_step=self.time_step+1, bump=bump, breeze=self.is_breeze(), stench=self.is_stench(), scream=False, glitter=self.is_glitter(), reward=reward, done=self.game_over)

        elif action == Action.GRAB:
            # Grab gold if present
            if self.is_glitter():
                self.agent_has_gold = True
            return Percept(time_step=self.time_step+1, bump=False, breeze=self.is_breeze(), stench=self.is_stench(), scream=False, glitter=self.is_glitter(), reward=-1, done=self.game_over)

        elif action == Action.SHOOT:
            # Shoot arrow 
            wumpus_orig_alive = self.wumpus_alive
            agent_has_arrow_orig = self.agent_has_arrow
            self.kill_attempt()
            wumpus_curr_alive = self.wumpus_alive
            agent_has_arrow_curr = self.agent_has_arrow
            self.agent_has_arrow = False
            if wumpus_orig_alive==True and wumpus_curr_alive == False:
                scream = True 
            else:
                scream = False
                
            if agent_has_arrow_orig == True and agent_has_arrow_curr == False:
                reward = -10
            else:
                reward = -1
            return Percept(time_step=self.time_step+1, bump=False, breeze=self.is_breeze(), stench=self.is_stench(), scream=scream, glitter=self.is_glitter(), reward=reward, done=self.game_over)

        elif action == Action.CLIMB:
            # Climb out of cave 
            if self.agent_location == Location(1,1) and self.allow_climb_without_gold==True and self.agent_has_gold==True:
                self.game_over = True
                reward = 999
            elif self.agent_location == Location(1,1) and self.allow_climb_without_gold==True and self.agent_has_gold==False:
                self.game_over = True
                reward = -1
            elif self.agent_location == Location(1,1) and self.allow_climb_without_gold==False and self.agent_has_gold==True:
                self.game_over = True
                reward = 999
            else:
                reward = -1
            return Percept(time_step=self.time_step+1, bump=False, breeze=self.is_breeze(), stench=self.is_stench(), scream=False, glitter=self.is_glitter(), reward=reward, done=self.game_over)

        
    # Visualize the game state
    def visualize(self):
        for y in range(WORLD_SIZE_y, 0, -1):
            line = '|'
            for x in range(1, WORLD_SIZE_x+1):
                loc = Location(x, y)
                cell_symbols = [' ', ' ', ' ', ' ']
                if self.is_agent_at(loc): cell_symbols[0] = 'A'
                if self.is_pit_at(loc): cell_symbols[1] = 'P'
                if self.is_wumpus_at(loc):
                    if self.wumpus_alive:
                        cell_symbols[2] = 'W'
                    else:
                        cell_symbols[2] = 'w'
                if self.is_gold_at(loc): cell_symbols[3] = 'G'
                for char in cell_symbols: line += char
                line += '|'
            print(line)

In [40]:
class NaiveAgent:
    
    def choose_action(self):
        # return a randomly chosen Action
        self.nextAction = Action(random.randint(0, 5))
        return self.nextAction
    
    def run(self, WORLD_SIZE_x, WORLD_SIZE_y, allow_climb_without_gold, pit_prob):
        env = Environment(WORLD_SIZE_x, WORLD_SIZE_y, allow_climb_without_gold, pit_prob)
        cumulative_reward = 0
        percept =  Percept(time_step=0, bump=False, breeze=env.is_breeze(), stench=env.is_stench(), scream=False, glitter=env.is_glitter(), reward=0, done=env.game_over)
        while not percept.done:
            env.visualize()
            print('Percept:', percept)
            self.choose_action()
            print()
            print('Action:', self.nextAction)
            print()
            percept = env.step(self.nextAction)
            cumulative_reward += percept.reward
        env.visualize()
        print('Percept:', percept)
        print('Cumulative reward:', cumulative_reward)

In [68]:
Action(random.choice([0,1,2,4]))

<Action.FORWARD: 2>

In [ ]:
class MovePlanningAgent:
    
    def choose_action(self):
        # return a randomly chosen Action except for Grab and Climb
        self.nextAction = Action(random.choice([0,1,2,4]))
        return self.nextAction
    
    def run(self, WORLD_SIZE_x, WORLD_SIZE_y, allow_climb_without_gold, pit_prob):
        env = Environment(WORLD_SIZE_x, WORLD_SIZE_y, allow_climb_without_gold, pit_prob)
        cumulative_reward = 0
        percept =  Percept(time_step=0, bump=False, breeze=env.is_breeze(), stench=env.is_stench(), scream=False, glitter=env.is_glitter(), reward=0, done=env.game_over)
        while not percept.done:
            env.visualize()
            print('Percept:', percept)
            self.choose_action()
            print()
            print('Action:', self.nextAction)
            print()
            percept = env.step(self.nextAction)
            cumulative_reward += percept.reward
        env.visualize()
        print('Percept:', percept)
        print('Cumulative reward:', cumulative_reward)

In [41]:
# Test 1
naive_agent_game = NaiveAgent()
naive_agent_game.run(WORLD_SIZE_x, WORLD_SIZE_y, True, 0.2)

|    |    |    |    |
|  W |   G|    |    |
| P  |    |    |    |
|A   | P  |    |    |
Percept: Percepts: Time Step=0, Bump=False, Breeze=True, Stench=False, Scream=False, Glitter=False, Reward=0, Done=False

Action: Action.SHOOT

|    |    |    |    |
|  W |   G|    |    |
| P  |    |    |    |
|A   | P  |    |    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=True, Stench=False, Scream=False, Glitter=False, Reward=-10, Done=False

Action: Action.SHOOT

|    |    |    |    |
|  W |   G|    |    |
| P  |    |    |    |
|A   | P  |    |    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=True, Stench=False, Scream=False, Glitter=False, Reward=-1, Done=False

Action: Action.LEFT

|    |    |    |    |
|  W |   G|    |    |
| P  |    |    |    |
|A   | P  |    |    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=True, Stench=False, Scream=False, Glitter=False, Reward=-1, Done=False

Action: Action.FORWARD

|    |    |    |    |
|  W |   G|    |    |
|AP  |    |    |    |
|

In [35]:
# Test 2
naive_agent_game = NaiveAgent()
naive_agent_game.run(WORLD_SIZE_x, WORLD_SIZE_y, True, 0.2)

| P  |    |    | P  |
|    |    | P  | P  |
|  W | P  |   G|    |
|A   | P  |    |    |
Percept: Percepts: Time Step=0, Bump=False, Breeze=True, Stench=True, Scream=False, Glitter=False, Reward=0, Done=False

Action: Action.SHOOT

| P  |    |    | P  |
|    |    | P  | P  |
|  W | P  |   G|    |
|A   | P  |    |    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=True, Stench=True, Scream=False, Glitter=False, Reward=-10, Done=False

Action: Action.FORWARD

| P  |    |    | P  |
|    |    | P  | P  |
|  W | P  |   G|    |
|    |AP  |    |    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=True, Stench=False, Scream=False, Glitter=False, Reward=-1001, Done=True
Cumulative reward: -1011


In [36]:
# Test 3
naive_agent_game = NaiveAgent()
naive_agent_game.run(WORLD_SIZE_x, WORLD_SIZE_y, True, 0.2)

| P  |    |    |    |
|    | P  |    |    |
|   G|  W |    |    |
|A   |    |    |    |
Percept: Percepts: Time Step=0, Bump=False, Breeze=False, Stench=False, Scream=False, Glitter=False, Reward=0, Done=False

Action: Action.LEFT

| P  |    |    |    |
|    | P  |    |    |
|   G|  W |    |    |
|A   |    |    |    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=False, Stench=False, Scream=False, Glitter=False, Reward=-1, Done=False

Action: Action.LEFT

| P  |    |    |    |
|    | P  |    |    |
|   G|  W |    |    |
|A   |    |    |    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=False, Stench=False, Scream=False, Glitter=False, Reward=-1, Done=False

Action: Action.LEFT

| P  |    |    |    |
|    | P  |    |    |
|   G|  W |    |    |
|A   |    |    |    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=False, Stench=False, Scream=False, Glitter=False, Reward=-1, Done=False

Action: Action.LEFT

| P  |    |    |    |
|    | P  |    |    |
|   G|  W |    |    |
|A 

In [37]:
# Test 4
naive_agent_game = NaiveAgent()
naive_agent_game.run(WORLD_SIZE_x, WORLD_SIZE_y, True, 0.2)

|    |    |    |  W |
|    |    |    |    |
|    | P  |    |   G|
|A   | P  |    |    |
Percept: Percepts: Time Step=0, Bump=False, Breeze=True, Stench=False, Scream=False, Glitter=False, Reward=0, Done=False

Action: Action.RIGHT

|    |    |    |  W |
|    |    |    |    |
|    | P  |    |   G|
|A   | P  |    |    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=True, Stench=False, Scream=False, Glitter=False, Reward=-1, Done=False

Action: Action.GRAB

|    |    |    |  W |
|    |    |    |    |
|    | P  |    |   G|
|A   | P  |    |    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=True, Stench=False, Scream=False, Glitter=False, Reward=-1, Done=False

Action: Action.FORWARD

|    |    |    |  W |
|    |    |    |    |
|    | P  |    |   G|
|A   | P  |    |    |
Percept: Percepts: Time Step=1, Bump=True, Breeze=True, Stench=False, Scream=False, Glitter=False, Reward=-1, Done=False

Action: Action.CLIMB

|    |    |    |  W |
|    |    |    |    |
|    | P  |    |   G|
|A 

In [43]:
# Test 5
naive_agent_game = NaiveAgent()
naive_agent_game.run(WORLD_SIZE_x, WORLD_SIZE_y, True, 0.2)

| P G|    |    |  W |
|    | P  |    |    |
|    |    | P  |    |
|A   |    |    |    |
Percept: Percepts: Time Step=0, Bump=False, Breeze=False, Stench=False, Scream=False, Glitter=False, Reward=0, Done=False

Action: Action.GRAB

| P G|    |    |  W |
|    | P  |    |    |
|    |    | P  |    |
|A   |    |    |    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=False, Stench=False, Scream=False, Glitter=False, Reward=-1, Done=False

Action: Action.CLIMB

| P G|    |    |  W |
|    | P  |    |    |
|    |    | P  |    |
|A   |    |    |    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=False, Stench=False, Scream=False, Glitter=False, Reward=-1, Done=True
Cumulative reward: -2


In [44]:
# Test 6
naive_agent_game = NaiveAgent()
naive_agent_game.run(WORLD_SIZE_x, WORLD_SIZE_y, False, 0)

|    |   G|    |    |
|    |    |    |    |
|    |    |  W |    |
|A   |    |    |    |
Percept: Percepts: Time Step=0, Bump=False, Breeze=False, Stench=False, Scream=False, Glitter=False, Reward=0, Done=False

Action: Action.FORWARD

|    |   G|    |    |
|    |    |    |    |
|    |    |  W |    |
|    |A   |    |    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=False, Stench=False, Scream=False, Glitter=False, Reward=-1, Done=False

Action: Action.LEFT

|    |   G|    |    |
|    |    |    |    |
|    |    |  W |    |
|    |A   |    |    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=False, Stench=False, Scream=False, Glitter=False, Reward=-1, Done=False

Action: Action.SHOOT

|    |   G|    |    |
|    |    |    |    |
|    |    |  W |    |
|    |A   |    |    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=False, Stench=False, Scream=False, Glitter=False, Reward=-10, Done=False

Action: Action.LEFT

|    |   G|    |    |
|    |    |    |    |
|    |    |  W |    

In [45]:
# Test 7
naive_agent_game = NaiveAgent()
naive_agent_game.run(WORLD_SIZE_x, WORLD_SIZE_y, False, 0)

|    |    |    |    |
|    |    |    |  W |
|    |    |    |    |
|A   |    |   G|    |
Percept: Percepts: Time Step=0, Bump=False, Breeze=False, Stench=False, Scream=False, Glitter=False, Reward=0, Done=False

Action: Action.RIGHT

|    |    |    |    |
|    |    |    |  W |
|    |    |    |    |
|A   |    |   G|    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=False, Stench=False, Scream=False, Glitter=False, Reward=-1, Done=False

Action: Action.SHOOT

|    |    |    |    |
|    |    |    |  W |
|    |    |    |    |
|A   |    |   G|    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=False, Stench=False, Scream=False, Glitter=False, Reward=-10, Done=False

Action: Action.RIGHT

|    |    |    |    |
|    |    |    |  W |
|    |    |    |    |
|A   |    |   G|    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=False, Stench=False, Scream=False, Glitter=False, Reward=-1, Done=False

Action: Action.RIGHT

|    |    |    |    |
|    |    |    |  W |
|    |    |    |    

In [48]:
# Test 8
naive_agent_game = NaiveAgent()
naive_agent_game.run(WORLD_SIZE_x, WORLD_SIZE_y, False, 0)

|    |    |    |    |
|  W |    |    |    |
|    |    |    |    |
|A   |   G|    |    |
Percept: Percepts: Time Step=0, Bump=False, Breeze=False, Stench=False, Scream=False, Glitter=False, Reward=0, Done=False

Action: Action.SHOOT

|    |    |    |    |
|  W |    |    |    |
|    |    |    |    |
|A   |   G|    |    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=False, Stench=False, Scream=False, Glitter=False, Reward=-10, Done=False

Action: Action.RIGHT

|    |    |    |    |
|  W |    |    |    |
|    |    |    |    |
|A   |   G|    |    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=False, Stench=False, Scream=False, Glitter=False, Reward=-1, Done=False

Action: Action.GRAB

|    |    |    |    |
|  W |    |    |    |
|    |    |    |    |
|A   |   G|    |    |
Percept: Percepts: Time Step=1, Bump=False, Breeze=False, Stench=False, Scream=False, Glitter=False, Reward=-1, Done=False

Action: Action.CLIMB

|    |    |    |    |
|  W |    |    |    |
|    |    |    |    |